# Assignment 4

Isaiah D. Murray (idm22@cornell.edu)

Daan van der Zwagg (dv239@cornell.edu)

#### In this assignment, we will conclude our analysis of whether the stop and frisk policy was racially discriminatory, but from a very different angle than our previous mapping analysis. We will rely heavily on logistic regression and regularized regression, as discussed in class. 

# Data processing (5 points)

First we need to do some data processing for consistency with previous of analysis of stop_and_frisk data. Load in the data "sqf_sample" and filter for stops between 2008 and 2012 (including both 2008 and 2012 in your sample). Filter for stops of white, Black, and Hispanic pedestrians using the suspect_race column. (5 points)


In [1]:
# importing necessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# read in csv
sqf_sample = pd.read_csv("sqf_sample.csv")

In [3]:
data = sqf_sample

In [4]:
# check shape
print (data.shape)

(408615, 89)


In [5]:
# check columns
print (data.columns)

Index(['year', 'date', 'time', 'precinct', 'xcoord', 'ycoord', 'serial',
       'radio_run', 'inside_outside', 'location_housing', 'observation_period',
       'suspected_crime', 'stop_length', 'identification', 'reason_explained',
       'others_stopped', 'arrested', 'arrested_reason', 'summons_issued',
       'officer_uniform', 'officer_verbal', 'officer_shield', 'frisked',
       'searched', 'extra_reports', 'force_hands', 'force_wall',
       'force_ground', 'force_drawn', 'force_pointed', 'force_baton',
       'force_handcuffs', 'force_pepper', 'force_other', 'stopped_bc_object',
       'stopped_bc_desc', 'stopped_bc_casing', 'stopped_bc_lookout',
       'stopped_bc_clothing', 'stopped_bc_drugs', 'stopped_bc_furtive',
       'stopped_bc_violent', 'stopped_bc_bulge', 'stopped_bc_other',
       'frisked_bc_suspected_crime', 'frisked_bc_weapons', 'frisked_bc_attire',
       'frisked_bc_actual_crime', 'frisked_bc_noncompliance',
       'frisked_bc_threats', 'frisked_bc_prior', 'friske

In [6]:
# preview the data
data.head()

,year,date,time,precinct,xcoord,ycoord,serial,radio_run,inside_outside,location_housing,...,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,found_gun,found_weapon,id,lat,lon
0,2009,2009-02-13,13:01,9.0,990071.0,204546.0,730.0,False,False,neither,...,6.166667,185.0,black,brown,medium,False,False,1603997,40.728106,-73.978998
1,2010,2010-07-02,22:11,77.0,996834.0,184061.0,6369.0,False,False,neither,...,5.666667,175.0,black,brown,medium,False,False,2422905,40.671873,-73.954636
2,2007,2007-05-18,23:28,71.0,1002197.0,182674.0,2053.0,False,True,neither,...,5.750000,175.0,black,brown,medium,False,False,705704,40.668056,-73.935306
3,2010,2010-01-20,19:20,73.0,1010719.0,186857.0,967.0,False,True,transit,...,5.666667,180.0,black,brown,heavy,False,False,2133381,40.679516,-73.904570
4,2012,2012-01-18,19:50,48.0,1014014.0,249325.0,334.0,False,False,neither,...,6.000000,175.0,black,brown,medium,False,False,3425345,40.850964,-73.892414


In [7]:
# filtering for year
data = data[data["year"]>=2008]
data = data[data["year"]<=2012]

In [8]:
#checking the data shape
data.shape

(293278, 89)

In [9]:
# making sure that there are in fact only years 2008 to 2012
print (data["year"].unique())

[2009 2010 2012 2008 2011]


In [10]:
# seeing the unique values in suspect_race
print (data["suspect_race"].unique())

['black' 'hispanic' 'asian' 'white' nan 'other' 'native american']


In [11]:
# filtering for race
selected = ['black', 'hispanic', 'white']
data = data[data["suspect_race"].isin(selected)]

In [12]:
# seeing the unique values in suspect_race
print (data["suspect_race"].unique())

['black' 'hispanic' 'white']


In [13]:
data.shape

(273235, 89)

In [14]:
data.head()

,year,date,time,precinct,xcoord,ycoord,serial,radio_run,inside_outside,location_housing,...,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,found_gun,found_weapon,id,lat,lon
0,2009,2009-02-13,13:01,9.0,990071.0,204546.0,730.0,False,False,neither,...,6.166667,185.0,black,brown,medium,False,False,1603997,40.728106,-73.978998
1,2010,2010-07-02,22:11,77.0,996834.0,184061.0,6369.0,False,False,neither,...,5.666667,175.0,black,brown,medium,False,False,2422905,40.671873,-73.954636
3,2010,2010-01-20,19:20,73.0,1010719.0,186857.0,967.0,False,True,transit,...,5.666667,180.0,black,brown,heavy,False,False,2133381,40.679516,-73.904570
4,2012,2012-01-18,19:50,48.0,1014014.0,249325.0,334.0,False,False,neither,...,6.000000,175.0,black,brown,medium,False,False,3425345,40.850964,-73.892414
5,2010,2010-11-22,22:35,83.0,1008499.0,191844.0,8631.0,False,False,neither,...,5.750000,170.0,black,brown,medium,False,False,2650242,40.693211,-73.912556


## Using regression to analyze the frisk decision. (30 points)

We will start by testing for racial discrimination in the decision to conduct a frisk after a stop: ie, whether Black and Hispanic pedestrians are more likely to be frisked (patted down for weapons) after they are stopped, controlling for other factors.

a. Using statsmodels, perform a logistic regression, using `frisked` as the dependent variable and `suspect_race` as the independent variable, to assess how the probability of being frisked after a stop varies by race. Write a few sentences interpreting the results, making sure to answer the following questions: which value of suspect_race is omitted from the regression coefficients? Which race groups are most likely to be frisked after being stopped? How do you interpret the magnitude and sign of the coefficients? How do you interpret their statistical significance and confidence intervals? (5 points)

In [15]:
#preview the data
print (data['frisked_bc_weapons'])

0         False
1         False
3         False
4         False
5         False
          ...  
408608    False
408609    False
408610    False
408611    False
408614    False
Name: frisked_bc_weapons, Length: 273235, dtype: bool


In [16]:
# converting data to dummies
dummies = pd.get_dummies(data['frisked_bc_weapons'])
print (dummies)

        False  True 
0           1      0
1           1      0
3           1      0
4           1      0
5           1      0
...       ...    ...
408608      1      0
408609      1      0
408610      1      0
408611      1      0
408614      1      0

[273235 rows x 2 columns]


In [51]:
# adding column to data
data['frisked_bc_weapons_dummies'] = dummies[1]
# assigning a column for frisked

In [52]:
# previewing the data
print (data['frisked_bc_weapons_dummies'].unique())
# 0 = False
# 1 = True

[0 1]


In [53]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race)'
logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())

Optimization terminated successfully.
         Current function value: 0.150585
         Iterations 7
                               Logit Regression Results                               
Dep. Variable:     frisked_bc_weapons_dummies   No. Observations:               273235
Model:                                  Logit   Df Residuals:                   273232
Method:                                   MLE   Df Model:                            2
Date:                        Mon, 08 Nov 2021   Pseudo R-squ.:               0.0005370
Time:                                09:58:03   Log-Likelihood:                -41145.
converged:                               True   LL-Null:                       -41167.
Covariance Type:                    nonrobust   LLR p-value:                 2.509e-10
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept 

### Answer

Which value of suspect race is omitted from the regression coefficients? 
Which race groups are most likely to be frisked after being stopped? 
How do you interpret the magnitude and sign of the coefficients? How do you interpret their statistical significance and confidence intervals?

From the logistic regression summary above, we can see that black is omitted from the regression coefficients, but that is only because it is noted as intercept and the other two races (i.e., Hispanic, and white) are evaluated with respect to the intercept. That being said, blacks are more likely than Hispanics and whites to be stopped and frisked for weapons. Looking at the coefficients, we see this. All the coefficients are negative meaning each race category is already negatively correlated with being stopped and frisked for weapons, but among these groups there is disparate impact. Looking at the coefficients, we see that blacks are more likely to be stopped and frisked, then Hispanics, and lastly whites. To quantify this further, by taking e^ ("each race coefficient"), we can better understand the impacts of race on being stopped and frisked for weapons. People who are stopped and frisked for weapons there is a .038 chance they are black, .034 they are Hispanic, and .031 they are white. The p-values for each of the race groups is less than .05 and are significant. The confidence intervals are narrow and from the large number of observations in that the results were based on, this is expected, and we can trust.


b. Now perform a linear regression instead of a logistic regression using the same formula. How is the interpretation of the coefficients similar or different in the two regressions? What are the advantages of each? (5 points)

In [54]:
#linear regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race)'
lin_regress = smf.ols(formula=str(f), data=data).fit()
print (lin_regress.summary())

#lin_reg=sm.OLS(ytrain,xtrain).fit()
#print(lin_reg.summary())

                                OLS Regression Results                                
Dep. Variable:     frisked_bc_weapons_dummies   R-squared:                       0.000
Model:                                    OLS   Adj. R-squared:                  0.000
Method:                         Least Squares   F-statistic:                     21.66
Date:                        Mon, 08 Nov 2021   Prob (F-statistic):           3.91e-10
Time:                                10:40:39   Log-Likelihood:                 76375.
No. Observations:                      273235   AIC:                        -1.527e+05
Df Residuals:                          273232   BIC:                        -1.527e+05
Df Model:                                   2                                         
Covariance Type:                    nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

### Answer

How is the interpretation of the coefficients similar or different in the two regressions? What are the advantages of each?

Both logistic and linear regression both show that blacks have the highest odds of being stopped and frisked for weapons compared to other race groups. Given the nature of the regressions, logistic regression provides a better interpretation of the data since we are trying to evaluate whether race groups have an impact on a binary outcome (stopped and frisked for weapons or NOT stopped and frisked for weapons). There is an inherent separation in the data that is best modeled by logistic regression and not linear regression. When groups are being compared to each other in a success / fail outcome, then logistic regression is best; when observing the collective correlation of features on the likelihood of an outcome, linear regression should be used.


c. The regression using only race as an independent variable is a good starting point, but it does not control for any other variables. What other variables do you think are important to control for, and why?  (3 points)

In [57]:
 # previewing the columns that could be incorporated
print (list(data.columns))

['year', 'date', 'time', 'precinct', 'xcoord', 'ycoord', 'serial', 'radio_run', 'inside_outside', 'location_housing', 'observation_period', 'suspected_crime', 'stop_length', 'identification', 'reason_explained', 'others_stopped', 'arrested', 'arrested_reason', 'summons_issued', 'officer_uniform', 'officer_verbal', 'officer_shield', 'frisked', 'searched', 'extra_reports', 'force_hands', 'force_wall', 'force_ground', 'force_drawn', 'force_pointed', 'force_baton', 'force_handcuffs', 'force_pepper', 'force_other', 'stopped_bc_object', 'stopped_bc_desc', 'stopped_bc_casing', 'stopped_bc_lookout', 'stopped_bc_clothing', 'stopped_bc_drugs', 'stopped_bc_furtive', 'stopped_bc_violent', 'stopped_bc_bulge', 'stopped_bc_other', 'frisked_bc_suspected_crime', 'frisked_bc_weapons', 'frisked_bc_attire', 'frisked_bc_actual_crime', 'frisked_bc_noncompliance', 'frisked_bc_threats', 'frisked_bc_prior', 'frisked_bc_furtive', 'frisked_bc_bulge', 'additional_report', 'additional_investigation', 'additional_p

In [91]:
# exploring unique values in columns
print (data['radio_run'].unique())

[False  True]


In [92]:
# testing the variable

#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race) + C(radio_run)'
logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())

Optimization terminated successfully.
         Current function value: 0.150515
         Iterations 7
                               Logit Regression Results                               
Dep. Variable:     frisked_bc_weapons_dummies   No. Observations:               273235
Model:                                  Logit   Df Residuals:                   273231
Method:                                   MLE   Df Model:                            3
Date:                        Mon, 08 Nov 2021   Pseudo R-squ.:                0.001002
Time:                                12:43:39   Log-Likelihood:                -41126.
converged:                               True   LL-Null:                       -41167.
Covariance Type:                    nonrobust   LLR p-value:                 8.914e-18
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept 

### Answer

When considering disparate impacts in stop and frisk for weapons it is important to consider race. There are other aspects of that could be used to help explain who is disproportionately stopped and frisked. This includes radio_run. If someone is already run on the radio, they are actively looking for someone of a certain description which could mean that if someone is stopped and frisked, there is a reason for that. By incorporating this, we can compare remove those cases where someone was stopped because they fit a description that was already broadcasted and not for discriminatory reasons (i.e., can the disproportionate stopping of blacks and Hispanics be explained through radio).


d. Run a logistic regression where you control for both race and for the "precinct" variable, which encodes the police precinct in which the stop occurred. Make sure to control for precinct as a categorical, not a numerical, variable, by writing it as C(precinct) in the regression formula - why is this important to do?

How do the race coefficients change, and what does that mean? How does the interpretation of this regression differ from the regression in which you only control for race? Make one argument in favor of reporting results controlling for location, and one argument against it. (5 points)

In [81]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race) + C(precinct)'

logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())


Optimization terminated successfully.
         Current function value: 0.148051
         Iterations 9
                               Logit Regression Results                               
Dep. Variable:     frisked_bc_weapons_dummies   No. Observations:               273235
Model:                                  Logit   Df Residuals:                   273157
Method:                                   MLE   Df Model:                           77
Date:                        Mon, 08 Nov 2021   Pseudo R-squ.:                 0.01736
Time:                                11:26:19   Log-Likelihood:                -40453.
converged:                               True   LL-Null:                       -41167.
Covariance Type:                    nonrobust   LLR p-value:                6.634e-248
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept 

### Answer

It is important to code for precinct as a categorical variable over a numerical variables because each precinct is not related linearly. Putting them into the logistic regression without the "C", will not render the correct output. The race coefficients change; the coefficients decrease for blacks and Hispanics, but increases for whites, when accounting for precinct. This proposes a different story.

#### For Reporting Location Results
When reporting this data accounting for precinct we see that whites are stopped and frisked for weapons more than other race groups. This explains some of the high rates of stopping and frisking people of color, nonetheless this brings about a new observation - that some precincts over others are more likely to stop and frisk. These precincts may also be places that are overly surveilled. This indicates that there is a need to focus on the disparate impacts across police precincts and investigate why this is so.


#### Not For Reporting Location Results
When reporting this data without accounting for precinct, we can see that blacks and Hispanics have higher odds of being stopped and frisked by police for weapons. This can show that there might be an issue with implicit bias among police officers.

#### Bringing These Together
I would choose to report both. Overall, we can see that blacks and Hispanics are stopped at higher rates than white people, but when accounting for precinct, this is no longer the story. However, there are precincts that have higher rates for police stop and frisk for weapons. A question that arises from this includes are stop and frisk for weapons motivated by implicit biases against Blacks and Hispanics, which then pushed for increased stop and frisk for weapons rates in majority Black and Hispanic police precincts.


In [80]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race)'
logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())

Optimization terminated successfully.
         Current function value: 0.150585
         Iterations 7
                               Logit Regression Results                               
Dep. Variable:     frisked_bc_weapons_dummies   No. Observations:               273235
Model:                                  Logit   Df Residuals:                   273232
Method:                                   MLE   Df Model:                            2
Date:                        Mon, 08 Nov 2021   Pseudo R-squ.:               0.0005370
Time:                                11:25:12   Log-Likelihood:                -41145.
converged:                               True   LL-Null:                       -41167.
Covariance Type:                    nonrobust   LLR p-value:                 2.509e-10
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept 

e. In a few sentences, explain why it is a BAD idea, conceptually, to control for the following variables if we are trying to assess whether the police racially discriminate in whom they frisk after a stop: a) "found.weapon", which encodes whether the frisk found a weapon, and b) "suspect.eye" and "suspect.hair", which encode the suspect's eye and hair color. (4 points)

### Answer

a.) It is a bad idea to control for "found weapon" because in doing so, it does not help explain whether different racial groups are disparately stopped and frisked for weapons, but instead, validates discriminatory stop and frisks if the suspect had a weapon.

b.) It would also be a bad idea to account for features such as eye color and hair because these visual features could account for some variation in the model, but these features are often associated with race. E.g., white people are more likely to have green/blue/etc. colored eyes, and blonde hair, while Blacks and Hispanics are likely to have darker colored hair and brown / dark brown eyes. The results here might be very similar to what we had before.


f. In a few sentences, explain the problem of omitted variable bias in this analysis, and how it would undermine the conclusions. (4 points)

### Answer
Depending on the story that someone wants to prove through the data, there can be a bias to include or not include certain variables from the analysis. As seen with the inclusion of "precinct" the results changed to show that actually whites are more likely to be stopped. Although this is true in that analysis, when we don't account for precinct, we see that blacks are at a higher risk than Hispanics and whites for being stopped and frisked for weapons.

In another case, someone who does not have domain expertise my unknowingly include or omit variables that could misrepresent what is going on. Suppose if someone does not include race in this analysis, they could replace that with hair color and show that people with darker hair colors are stopped more frequently than those who do not. The conclusion here would report that it is hair color that explains the basis for discrimination for police.

With a topics as controversial as racial discrimination, no one wants to be labeled as racially discriminatory, so they will try to shape an analysis to disprove that with the methods in data analysis.




g. In a few sentences, explain why only examining whether someone is frisked after a stop might fail to provide a full picture of discrimination in the stop and frisk policy. (4 points)

### Answer
Examining whether someone is frisked after a stop fails to provide a full picture of discrimination because the reason for initial the stop could be motivated by a myriad of reasons, which then called for a frisk. This is different from stop and frisk in that the motivation for it was mere suspicion and not a defined violation.



## Outcome analysis using regularized regression (65 points)

Because of the issues with omitted variables in analyses like the one above, *outcome* tests are often used: these look not at the rate at which a decision is made (like the decision to frisk), but at the outcome of the decision (for example, if the frisk is conducted to find a weapon, does it actually find one?) Now we will use an outcome-style analysis. Specifically, we will fit a machine learning model to predict the probability that each stop which was conducted on suspicion the pedestrian possessed a weapon actually finds a weapon. Stops which are very unlikely to find a weapon arguably violate the Fourth Amendment, which prohibits unreasonable searches; if such stops disproportionately occur of certain race groups, the policy may violate the Fourteenth Amendment, which prohibits racial discrimination. 

a. In this portion of this analysis, you will be using a smaller version of the data to speed up model fitting. Load in "small_sqf_sample". As before, we need to do some data processing for consistency with previous of analysis of this data. Load in the data and filter for stops between 2008 and 2012 (including both 2008 and 2012); filter for stops of white, Black, and Hispanic pedestrians using the suspect_race column; and filter for stops conducted on suspicion of criminal posession of a weapon (ie, suspected_crime == 'cpw'). (5 points)

In [33]:
# loading the data in
sample_data = pd.read_csv("small_sqf_sample.csv")

In [34]:
# filtering for year
sample_data = sample_data[sample_data["year"]>=2008]
sample_data = sample_data[sample_data["year"]<=2012]
print (data["year"].unique())

# filtering for race
selected = ['black', 'hispanic', 'white']
sample_data = sample_data[sample_data["suspect_race"].isin(selected)]
# seeing the unique values in suspect_race
print (sample_data["suspect_race"].unique())

# filtering for suspected crime
sample_data = sample_data[sample_data["suspected_crime"]=="cpw"]
print (sample_data["suspected_crime"].unique())

[2009 2010 2012 2008 2011]
['black' 'white' 'hispanic']
['cpw']



b. We will be fitting the regression model 

`found_weapon ~ C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex + ADDITIONAL_CIRCUMSTANCE_COLUMNS` 

where ADDITIONAL_CIRCUMSTANCE_COLUMNS are any columns that begin with "stopped_bc" or "additional_" besides "additional_other" and "stopped_bc_other". You can get these columns by running

`ADDITIONAL_CIRCUMSTANCE_COLUMNS = [a for a in d.columns if ('stopped_bc' in a or 'additional_' in a) and a not in (['additional_other', 'stopped_bc_other'])]`

You should have 18 additional columns. These columns provide more information about the circumstances of the stop, and we include them for consistency with the original analysis and because they turn out to be important for predictive performance. 

Drop any rows with missing values in any of the variables you need. 

Now, we need to put the data into a format which sklearn can use later - ie, numpy arrays. Do this with "patsy" library and the "dmatrix" function. You can call dmatrix as follows:

`sqf_X = patsy.dmatrix('C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex +' + '+'.join(ADDITIONAL_CIRCUMSTANCE_COLUMNS),sqf_data, return_type='dataframe')`

and it will return a dataframe on which you can fit a regression model. The first argument to dmatrix is the formula that you want to use to make the dataframe; the second argument gives patsy the data you want to use; return_type='dataframe' ensures that you get a dataframe, not a patsy object which is hard to use.

Look at the output of dmatrix and explain what the columns mean. Why can't we just pass the columns from the original dataframe directly into the sklearn function? (8 points)


In [41]:
#create the additional columns
ADDITIONAL_CIRCUMSTANCE_COLUMNS = [a for a in sample_data.columns if ('stopped_bc' in a or 'additional_' in a) and a not in (['additional_other', 'stopped_bc_other'])]

In [42]:
#check the len of the other ADDITIONAL_CIRCUMSTANCE_COLUMNS
print (len(ADDITIONAL_CIRCUMSTANCE_COLUMNS))
print (ADDITIONAL_CIRCUMSTANCE_COLUMNS)


18
['stopped_bc_object', 'stopped_bc_desc', 'stopped_bc_casing', 'stopped_bc_lookout', 'stopped_bc_clothing', 'stopped_bc_drugs', 'stopped_bc_furtive', 'stopped_bc_violent', 'stopped_bc_bulge', 'additional_report', 'additional_investigation', 'additional_proximity', 'additional_evasive', 'additional_associating', 'additional_direction', 'additional_highcrime', 'additional_time', 'additional_sights']


In [38]:
#creating a new data frame with desired columns
desired_columns=sample_data[["found_weapon", "precinct", "suspect_race", "location_housing", "year", "suspect_age",
                             "suspect_height", "suspect_weight", "suspect_sex", 'stopped_bc_object', 'stopped_bc_desc', 
                             'stopped_bc_casing', 'stopped_bc_lookout', 'stopped_bc_clothing', 'stopped_bc_drugs', 
                             'stopped_bc_furtive', 'stopped_bc_violent', 'stopped_bc_bulge', 'additional_report', 'additional_investigation', 
                             'additional_proximity', 'additional_evasive', 'additional_associating', 'additional_direction',
                             'additional_highcrime', 'additional_time', 'additional_sights']]

desired_data = desired_columns.dropna()
print (desired_data.shape)

(36056, 27)


In [ ]:
sqf_X = patsy.dmatrix('C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex +' + '+'.join(ADDITIONAL_CIRCUMSTANCE_COLUMNS),sqf_data, return_type='dataframe')

c. As discussed in class, when fitting machine learning models, you should always divide the dataset into a train, val, and test set. Randomly divide the filtered, processed data into three pieces - the train set (60%), the val set (20%) and the test set (20%). (5 points)

d. We will be training a regularized logistic regression model to predict the outcome. When using many machine learning models, including regularized logistic regression, it is important to preprocess the input features so they are all on the same scale, for reasons discussed in class. In this case, we will take each column in the input data, subtract its mean, and divide by its standard deviation. This makes it so each column of the data has mean 0 and standard deviation 1. 

When scaling the data, it is important to compute the scaling using only the train set, as shown in class. The reason is that we are pretending that the train data is all we have access to to fit our model fitting pipeline, so we cannot "peek" at the validation or test sets to generate our scaling. Use sklearn's StandardScaler (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to fit a scaling transform on the train set and transform the train set (you can use the fit_transform method on the train set).Then apply the fitted StandardScaler model to the validation and test sets as well (using the transform --- not the fit --- method). (Look at the notebook we went through in class on regularization and lasso if you are confused.) The transformed datasets are the final datasets you will feed into your logistic regression model. (5 points)

e. Uisng sklearn.linear_model.LogisticRegression, fit a model on the train set to predict found_weapon. Set the "penalty" argument to "none" so that the model will not use any regularization; this corresponds to fitting a regular logistic regression model. If you get a `ConvergenceWarning`, increase the number of iterations using the `max_iter` argument; this means the model optimization needs more iterations to converge. (Look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) if you are uncertain which arguments to use!)

A standard measure of predictive performance for binary outcome variables like found_weapon is AUC. Higher values of AUC are better; an AUC of 1 means that the model is perfectly predicting the outcome; an AUC of 0.5 means that it is predicting it only as well as random chance. Report the AUC of the fitted model (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) on both the train and validation sets. What is the problem with using accuracy as a metric for this task? How does the train set AUC differ from the validation AUC, and does this make sense? (7 points)

f. Our logistic regression model is not using any regularization and appears to be overfitting. We will try to use regularization to reduce overfitting. You will be fitting an L1-penalized logistic regression model using code that looks something like

`LogisticRegression(C=sparsity_param, penalty='l1', solver='liblinear')`

(the "l1" specifies that we're using an L1 penalty, as discussed in class; the "liblinear" solver is an optimizer that works with the L1 penalty. See the logistic regression documentation for more details.) 

Increase and decrease the amount of regularization using different values of the C parameter, searching logarithmically over at least 20 values in the range from 1e-2 to 1. (Note: we do not recommend defining a variable named "C" in your code, because this may cause weird patsy issues, since patsy also uses "C". Give the variable another name. Sorry! I complained to the patsy people.) 

Print out the train set, val set, and test set AUC for each value of of the regularization parameter. Make a plot where the x-axis is the regularization parameter and the y-axis is AUC, with one line for train AUC, one line for val AUC, and one line for test AUC (use plt.semilogx to plot the lines so the x-axis will be logarithmic, making it easier to see the plot). Comment on the trends. Do you see evidence of overfitting? Explain. For the rest of this assignment, use the model with the highest AUC on the validation set. (10 points)

In a full analysis, it would make sense to play with other aspects of the model as well: for example, you could try using other forms of regularization (like L1 vs L2) or other classification algorithms besides logistic regression. The basic pattern, though, would be the same: fit the model on the train set, choose models on the val set, and once you've chosen your best model, assess your results (once!) on the test set.

g. Assess model *calibration* on the test set. This checks whether the model's predicted probabilities line up with the true probabilities, and is important to assess here because we will be analyzing the model's predicted probabilities. To assess calibration, take the 10% of rows of the test set with the highest model predictions and compare the mean model predicted probability (ie, the output of `predict_proba`) to the actual mean of the outcome variable. Repeat for the next 10% of rows, and for all 10% groups. Make a plot comparing the model predicted probabilities on each 10% group to the actual outcomes (mean predicted probability on the x-axis and mean actual outcome on the y-axis). You should end up with a plot with 10 points, one for each 10% group. Plot the line y=x so you can see how well the predicted probabilities line up with the actual probabilities - ideally, your points will lie on the line or close to it. (10 points)

Calibration is an issue for many machine learning models, including deep learning models, so this is always a good thing to check. In a full analysis, it would make sense to check calibration (and AUC) for subgroups as well (eg, each race group).

h. Using the test set, compute the fraction of observations which have lower than a 2% model-predicted probability of finding a weapon for Black, Hispanic, and white pedestrians. Stops below this threshold are extremely unlikely to have resulted in finding a weapon, arguably violating the Fourth Amendment. Repeat this for 20 thresholds evenly spaced between 1% and 5%. Make a graph where the x-axis is the threshold, and the y-axis is the fraction of stops falling below that threshold, with one line for each race group. Explain what you observe. Do you think this graph provides evidence for Fourteenth Amendment violations (racial discrimination)? (10 points)

i. The analysis you have performed in the second part of this assignment is very similar to the analysis in [this paper](https://5harad.com/papers/stop-and-frisk.pdf). Read the paper and write a few sentences about their main conclusions. (5 points)